# Build your first agent!



# Environment


In [1]:
!pip install -qU google-adk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229

In [23]:
import os
PROJECT_ID = "your-project-id"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [3]:
from google.colab import auth
auth.authenticate_user()

In [135]:
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google import adk

import warnings
import logging
from pprint import pprint
import json
import time

# Utils

In [44]:
def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()

    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()

        try:
            agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))
        except AttributeError as e:
            print(f"Error parsing event content: {e}")
            continue

        if is_final_response:
            final_response = event.content.parts[0].text if event.content.parts else "No content available"
            elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
            print(f'>>> Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-" * 30)
        elif function_calls:
            print('+++ Function Calls:')
            for function_call in function_calls:
                print(f"Function Name: {function_call.name}, Args: {function_call.args}")
        elif function_responses:
            print('--- Function Responses:')
            for function_response in function_responses:
                response_details = function_response.response
                recommended_list = list(response_details.values()) if response_details else []
                print(f"Function Name: {function_response.name}")
                print(f"Function Results: {json.dumps(recommended_list)}")
        else:
            print('No function calls or responses available.')
            print(f"Agent Response: {agent_res}")

    elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
    print(f"Total elapsed time: {elapsed_time_ms} ms")

# Concepts: AGENT


In [60]:
AGENT_MODEL = "gemini-2.5-flash" # the model used by the agent

In [61]:
# My first agent
root_agent = Agent(
    model=AGENT_MODEL,
    name="PhysicsPal",
    instruction="""
    Explain the physics concepts in plain English so a beginner can understand
        """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=1000,
    ),
    description="Agent that explains concepts from Physics",
)

In [47]:
# Constant
APP_NAME = "physics_pal_app" # name of your agent app
USER_ID = "user12345"         # the user id that is interacting with the agent - used for a new conversation with the agent

### Setup Runner and Session Service

- To manage conversations and execute the agent, we need two more components:

**SessionService:** Responsible for managing conversation history and state for different users and sessions. The InMemorySessionService is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged. We'll explore state persistence more in Step 4.

**Runner:** The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the SessionService, and yields events representing the progress of the interaction.



In [48]:
# define the session service
session_service = InMemorySessionService()

In [49]:
# define runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

In [50]:
#create session
session = await session_service.create_session(
     app_name=APP_NAME,
     user_id=USER_ID,
)

In [51]:
user_input = types.Part(text="hi")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

>>> Final Response (1955.184 ms):
Hello there! I'm PhysicsPal, and I'm ready to help you understand some fascinating physics concepts.

What's on your mind today? Is there anything in particular you're curious about, or would you like me to start with something foundational? I'm here to explain things in plain English, so don't hesitate to ask anything!
------------------------------
Total elapsed time: 1963.196 ms


In [52]:
user_input = types.Part(text="What is inertia?")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

>>> Final Response (6869.842 ms):
Ah, inertia! That's a fantastic place to start because it's a really fundamental concept in physics, and you experience it every single day without even realizing it.

In the simplest terms, **inertia is the natural tendency of any object to resist changes in its state of motion.**

Let's break that down:

1.  **"Resist changes"**: This is the key part.
    *   If an object is **sitting still**, inertia wants it to *stay sitting still*. It's "lazy" and doesn't want to start moving.
    *   If an object is **moving**, inertia wants it to *keep moving in the same direction at the same speed*. It's equally "lazy" and doesn't want to stop or change its path.

Think of it like an object's "stubbornness" or "laziness" when it comes to changing what it's already doing.

**Here are a few everyday examples to help it click:**

*   **When you're in a car and it suddenly brakes:** You lurch forward, right? That's your body's inertia. Before the car braked, your b

In [53]:
user_input = types.Part(text="What is velocity?")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

>>> Final Response (6996.016 ms):
Great question! Velocity is a super important concept, and it's closely related to something you probably already understand: speed.

Think of it this way:

**Speed tells you *how fast* something is moving.**
*   "I'm driving at 60 miles per hour."
*   "That runner is doing 10 meters per second."

**Velocity tells you *how fast* something is moving AND *in what direction*.**

So, **velocity is essentially "speed with a direction."**

Let's use an example to make this super clear:

Imagine you're talking about a car:

*   **Speed:** If I say, "The car is traveling at 50 miles per hour," that's its speed. You know *how fast* it's going.
*   **Velocity:** If I say, "The car is traveling at 50 miles per hour *north*," that's its velocity. Now you know both *how fast* it's going and *where* it's headed.

**Why is the direction so important?**

In physics, direction matters a lot!

*   If two cars are both going 50 mph, but one is going north and the other i

In [54]:
user_input = types.Part(text="Do you have the list of all questions that I asked you in this session?")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

>>> Final Response (2784.675 ms):
Yes, I do! Here's a list of the questions you've asked me so far in this session:

1.  "hi"
2.  "What is inertia?"
3.  "What is velocity?"
4.  "Do you have the list of all questions that I asked you in this session?"
------------------------------
Total elapsed time: 2793.606 ms


In [55]:
session_tmp = await session_service.get_session(user_id=USER_ID, session_id=session.id, app_name=APP_NAME)
session_tmp

Session(id='51dfa0f6-906e-44f3-b64b-45e5ea0434bf', app_name='physics_pal_app', user_id='user12345', state={}, events=[Event(content=Content(
  parts=[
    Part(
      text='hi'
    ),
  ],
  role='user'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-bf422707-da94-47d5-aad6-5e7df0446ded', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='315bdf13-2fa2-4ff0-91b3-640add79308e', timestamp=1752767003.726153), Event(content=Content(
  parts=[
    Part(
      text="""Hello there! I'm PhysicsPal, and I'm ready to help you understand some fascinating physics concepts.

What's on your mind today? Is there anything in particular you're curious about, or would you like me to start with something foundational? I'm h

In [56]:
session_tmp.events[1]

Event(content=Content(
  parts=[
    Part(
      text="""Hello there! I'm PhysicsPal, and I'm ready to help you understand some fascinating physics concepts.

What's on your mind today? Is there anything in particular you're curious about, or would you like me to start with something foundational? I'm here to explain things in plain English, so don't hesitate to ask anything!"""
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=74,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=74
    ),
  ],
  prompt_token_count=46,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=46
    ),
  ],
  thoughts_token_count=37,
  total_token_count=157,
  traffic_type=<TrafficType.ON_DE

## Concept: Tools

In [160]:
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}


In [161]:
# Our second agent and it can can use tools!

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather, adk.tools.preload_memory_tool.PreloadMemoryTool()], # Pass the function directly
)

In [162]:
# Constant
WEATHER_APP_NAME = "weather_app" # name of your agent app
WEATHER_USER_ID = "user42"         # the user id that is interacting with the agent - used for a new conversation with the agent

In [163]:
# define the session and memory service
weather_session_service = InMemorySessionService()
weather_memory_service = InMemoryMemoryService()

In [164]:
# create the session and the runner
weather_session = await weather_session_service.create_session(
                                app_name=WEATHER_APP_NAME,
                                user_id=WEATHER_USER_ID
                            )
weather_runner = Runner(agent=weather_agent,
                        app_name=WEATHER_APP_NAME,
                        session_service=weather_session_service,
                        memory_service=weather_memory_service)



In [165]:
user_input = types.Part(text="hi I am planning a trip to Europe")

events = weather_runner.run(user_id=weather_session.user_id, session_id=weather_session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

>>> Final Response (1415.301 ms):
That sounds exciting! Which city in Europe are you planning to visit? I can help you with the weather there.
------------------------------
Total elapsed time: 1424.618 ms


In [166]:
# Get the completed session
completed_session1 = await weather_runner.session_service.get_session(app_name=WEATHER_APP_NAME, user_id=WEATHER_USER_ID, session_id=weather_session.id)

# Add this session's content to the Memory Service
print("\n--- Adding Session 1 to Memory ---")
memory_service = await weather_memory_service.add_session_to_memory(completed_session1)
print("Session added to memory.")


--- Adding Session 1 to Memory ---
Session added to memory.


In [167]:
# create a second session for the user
weather_session_2 = await weather_session_service.create_session(
                                app_name=WEATHER_APP_NAME,
                                user_id=WEATHER_USER_ID
                            )

In [169]:
input = 'Do you remember anything about our last conversation?' # @param {type:"string"}
user_input = types.Part(text=input)

events = weather_runner.run(user_id=weather_session_2.user_id,
                            session_id=weather_session_2.id,
                            new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

>>> Final Response (1579.909 ms):
Yes, I do! In our last conversation, you mentioned planning a trip to Europe, and I asked which city you were visiting so I could help you with the weather there.
------------------------------
Total elapsed time: 1584.588 ms


In [170]:
# Get the completed session
completed_session1 = await weather_runner.session_service.get_session(app_name=WEATHER_APP_NAME, user_id=WEATHER_USER_ID, session_id=weather_session.id)

# Add this session's content to the Memory Service
print("\n--- Adding Session 1 to Memory ---")
memory_service = await weather_memory_service.add_session_to_memory(completed_session1)
print("Session added to memory.")


--- Adding Session 1 to Memory ---
Session added to memory.


In [171]:
input = 'How is the weather in London' # @param {type:"string"}
user_input = types.Part(text=input)

events = weather_runner.run(user_id=weather_session.user_id, session_id=weather_session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

--- Tool: get_weather called for city: London ---+++ Function Calls:
Function Name: get_weather, Args: {'city': 'London'}

--- Function Responses:
Function Name: get_weather
Function Results: ["success", "It's cloudy in London with a temperature of 15\u00b0C."]
>>> Final Response (2804.541 ms):
It's cloudy in London with a temperature of 15°C.
------------------------------
Total elapsed time: 2808.854 ms


In [172]:
# Get the completed session
completed_session1 = await weather_runner.session_service.get_session(app_name=WEATHER_APP_NAME, user_id=WEATHER_USER_ID, session_id=weather_session.id)

# Add this session's content to the Memory Service
print("\n--- Adding Session 1 to Memory ---")
memory_service = await weather_memory_service.add_session_to_memory(completed_session1)
print("Session added to memory.")


--- Adding Session 1 to Memory ---
Session added to memory.
